In [97]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform



In [98]:
df = pd.read_csv("datasett.csv")
df.head




<bound method NDFrame.head of        manuell_klassifisering  ansatteid_prediksjon  testsett  \
0                         NaN                   NaN         0   
1                         NaN                   NaN         0   
2                         NaN                   NaN         0   
3                         NaN                   NaN         0   
4                         NaN                   NaN         0   
...                       ...                   ...       ...   
60848                     NaN                   NaN         0   
60849                     NaN                   NaN         0   
60850                     NaN                   NaN         0   
60851                     NaN                   NaN         0   
60852                     NaN                   NaN         0   

                                      bransjetekst_a_w  fylkesnr  komm_nr  \
0                          BYGGE- OG ANLEGGSVIRKSOMHET        11     1108   
1                VAREHANDEL, REPARA

In [99]:
f_labeled = df[df["manuell_klassifisering"].notnull()]
#f_labeled.head
t_labeld = df[df.testsett==1]


In [100]:
# 1. Les inn data og filtrer de med manuell klassifisering
df = pd.read_csv("datasett.csv")
df_labeled = df[df["manuell_klassifisering"].notnull()]

# 2. Del i X og y
X = df_labeled.drop(columns=["manuell_klassifisering"])
y = df_labeled["manuell_klassifisering"]
X_test = df_labeled.drop(columns=["manuell_klassifisering"])

# 3. Konverter alle object-kolonner i X til category
for col in X.select_dtypes(include="object").columns:
    X[col] = X[col].astype("category")

# 4. Splitt data
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# 5. Initialiser og tren modell
model = XGBRegressor(enable_categorical=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [101]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [102]:
y_pred = model.predict(X_valid)

In [103]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

print(f"MSE: {mse:.3f}")
print(f"R²: {r2:.3f}")


MSE: 0.114
R²: 0.445


In [104]:
#print(y_valid)
#print(y_pred)
print(np.round(y_pred, 1))

[ 0.9  0.3 -0.   0.3  0.3  0.1  0.5  0.   1.   0.4  0.7  1.   0.3  0.7
 -0.1  0.7  0.6  1.   0.8  0.9  0.9  1.   0.   0.7  0.3  0.1  0.7  0.3
  0.5  0.1 -0.   0.5  1.   0.9]


In [105]:

numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_valid_scaled = X_valid.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_valid_scaled[numeric_cols] = scaler.transform(X_valid[numeric_cols])
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_valid_scaled)



c:\Users\nicer\anaconda3\envs\in3050\Lib\site-packages\sklearn\utils\extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\nicer\anaconda3\envs\in3050\Lib\site-packages\sklearn\utils\extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\nicer\anaconda3\envs\in3050\Lib\site-packages\sklearn\utils\extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [106]:
param_dist = {
    "n_estimators": randint(100, 500),               # number of trees
    "max_depth": randint(3, 12),                     # tree depth
    "learning_rate": uniform(0.01, 0.3),             # shrinkage
    "subsample": uniform(0.6, 0.4),                  # fraction of samples used
    "colsample_bytree": uniform(0.6, 0.4),           # fraction of features used
    "gamma": uniform(0, 5),                          # min loss reduction to split
    "reg_alpha": uniform(0, 5),                      # L1 regularization
    "reg_lambda": uniform(0, 5),                     # L2 regularization
}


random_search = RandomizedSearchCV(
    estimator=XGBRegressor(enable_categorical=True),
    param_distributions=param_dist,
    n_iter=30,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_valid_scaled, y_valid) #skal det vere train eller validation data?

print("Best parameters:", random_search.best_params_)
print("Best MSE:", -random_search.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'colsample_bytree': 0.6661067756252009, 'gamma': 0.07818203370596966, 'learning_rate': 0.1370204442119109, 'max_depth': 3, 'n_estimators': 235, 'reg_alpha': 0.027610585618011996, 'reg_lambda': 4.0773071422741705, 'subsample': 0.8827429375390468}
Best MSE: 0.14176499425610334


In [107]:
best_model = random_search.best_estimator_
print(best_model)
y_pred = best_model.predict(X_train_scaled)
best_model = random_search.best_estimator_


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6661067756252009, device=None,
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, feature_types=None, gamma=0.07818203370596966,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1370204442119109,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=235, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


In [108]:
test_set = df[df["testsett"] == 1].drop(columns=["manuell_klassifisering"], errors='ignore')
test_set_scaled = test_set.copy()
test_set_scaled[numeric_cols] = scaler.transform(test_set[numeric_cols])

# Gjør object-kolonner til category
for col in test_set_scaled.select_dtypes(include="object").columns:
    test_set_scaled[col] = test_set_scaled[col].astype("category")

# Prediker

prediksjoner = best_model.predict(test_set_scaled)
print(prediksjoner)
df.loc[df["testsett"] == 1, "ansatteid_prediksjon"] = prediksjoner

[0.5873499  0.32522613 0.6441915  0.70177245 0.5343515  0.67648137
 0.73611015 0.46911836 0.54902667 0.7937383  0.27331394 0.47785684
 0.61467385 0.6653431  0.7287166  0.1331843  0.50131327 0.3487024
 0.77654207 0.35563836 0.06680953 0.35194224 0.230933   0.51648974
 0.9814565  0.0244791  0.7114381  0.80482036 0.49624515 0.8082649
 0.8058565  0.60192287 0.9008049  0.8926379  0.5304329  0.23027764
 0.68917614 0.59626925 0.5442746  0.6743921  0.56037664 0.37462682
 0.69874364 0.20676215 0.79704845 0.30698282 0.63381636 0.38485304
 0.5257959  0.6740849 ]


In [109]:
print(df.loc[df.testsett==1,"ansatteid_prediksjon"].to_json())

{"5":0.5873498917,"67":0.3252261281,"495":0.6441915035,"905":0.7017724514,"992":0.5343515277,"998":0.6764813662,"2679":0.7361101508,"2764":0.4691183567,"3334":0.5490266681,"3936":0.7937383056,"4117":0.2733139396,"4159":0.4778568447,"4232":0.6146738529,"4637":0.6653431058,"4831":0.7287166119,"5114":0.1331842989,"5671":0.5013132691,"6787":0.3487024009,"7703":0.7765420675,"9698":0.355638355,"10224":0.0668095276,"10748":0.3519422412,"14054":0.2309329957,"14471":0.5164897442,"17052":0.9814565182,"20151":0.0244790986,"26501":0.7114381194,"26858":0.8048203588,"27078":0.4962451458,"27693":0.8082649112,"27816":0.8058565259,"29574":0.6019228697,"30379":0.9008048773,"33279":0.8926379085,"33304":0.5304328799,"33852":0.2302776426,"36468":0.6891761422,"43648":0.5962692499,"43877":0.5442746282,"45341":0.6743921041,"45956":0.5603766441,"48032":0.3746268153,"48803":0.6987436414,"50905":0.2067621499,"51410":0.7970484495,"52885":0.3069828153,"55170":0.6338163614,"55977":0.3848530352,"57635":0.525795877,"